# Dotlas Odyssey 🏞 [40 points]

> Data Engineering Assignment

> `v1.0` Updated: Sep 2 2023

Greetings Traveller,

You have embarked on a mighty Odyssey, a journey filled with challenges, mysteries, and opportunities for growth. Your mission, should you choose to accept it, will involve navigating uncharted territories, solving complex problems, and contributing to a mission that will redefine our future. We believe that with your skills, passion, and determination, you can contribute significantly to this mission.

In this notebook, you will find a series of assignments designed to test your skills, stretch your capabilities, and ultimately, prepare you for the journey ahead. Each task is a stepping stone towards your ultimate goal, and completing them will not only bring you closer to becoming a part of the Dotlas family but also to making a meaningful impact on the world.

Remember, the journey of a thousand miles begins with a single step. So, gear up, stay positive, and embrace the adventure that awaits you.

---
<img src="https://dotlas-website.s3.eu-west-1.amazonaws.com/images/github/banner.png" width="750px" alt="dotlas">

> You are welcome to make your solution for the Dotlas Odyssey public as open-source and add it to your portfolio of projects, but only after all candidates have completed the assignment, or the position has been filled. Sharing your solution publicly while the assignment process is ongoing will result in disqualification from the selection process.

## Tools & Technologies 🪛

![Python](https://img.shields.io/badge/python-3670A0?style=for-the-badge&logo=python&logoColor=ffdd54)
![Anaconda](https://img.shields.io/badge/Anaconda-%2344A833.svg?style=for-the-badge&logo=anaconda&logoColor=white)
![Jupyter Notebook](https://img.shields.io/badge/jupyter-%23FA0F00.svg?style=for-the-badge&logo=jupyter&logoColor=white)
![Pandas](https://img.shields.io/badge/pandas-%23150458.svg?style=for-the-badge&logo=pandas&logoColor=white)
![Spark](https://img.shields.io/badge/Apache%20Spark-E25A1C.svg?style=for-the-badge&logo=Apache-Spark&logoColor=white)
![Matplotlib](https://img.shields.io/badge/Matplotlib-%23ffffff.svg?style=for-the-badge&logo=Matplotlib&logoColor=black)
![Numpy](https://img.shields.io/badge/NumPy-013243.svg?style=for-the-badge&logo=NumPy&logoColor=white)
![Plotly](https://img.shields.io/badge/Plotly-%233F4F75.svg?style=for-the-badge&logo=plotly&logoColor=white)
![AWS](https://img.shields.io/badge/AWS-%23FF9900.svg?style=for-the-badge&logo=amazon-aws&logoColor=white)

- This exercise will be carried out using the [Python](https://www.python.org/) programming language and will rely hevily on the [Pandas](https://pandas.pydata.org/) library for data manipulation.
- You are also free to use [Polars](https://www.pola.rs/), [Dask](https://www.dask.org/) or [Spark](https://spark.apache.org/docs/latest/api/python/index.html) if you do not want to use Pandas.
- You may use any of [Matplotlib](https://matplotlib.org/), [Seaborn](https://seaborn.pydata.org/) or [Plotly](https://plotly.com/python/) packages for data visualization.
- We will be using [Jupyter notebooks](https://jupyter.org/) to run Python code in order to view and interact better with our data and visualizations.
- You are free to use [Google Colab](https://colab.research.google.com/) which provides an easy-to-use Jupyter interface.
- When not in Colab, it is recommended to run this Jupyter Notebook within an [Anaconda](https://continuum.io/) environment
- You can use any other Python packages that you deem fit for this project.

> ⚠ **Ensure that your Python version is 3.9 or higher**

![](https://upload.wikimedia.org/wikipedia/commons/1/1b/Blue_Python_3.9_Shield_Badge.svg)

In [24]:
# YOUR CODE HERE
# Import necessary libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import ast
import csv
import requests
import time
import threading
from datetime import datetime
from requests import Timeout
from requests import RequestException


## Section 1: Extract 🚰 [5]

<img src="https://media.giphy.com/media/8rEB2xzZcZDnBegHFS/giphy.gif" width="250px" alt="extract">

You have to extract 4 datasets. Each dataset contains partial information about **restaurants in the San Francisco Bay Area, California, US**. The datasets are as follows:
1. **Delta**: A pipe delimited csv file that contains restaurants that offer food-delivery
2. **Oscar**: A raw binary file (pickled) that contains restaurants that take reservations, and other high-end places
3. **Tango**: A json file that contains all kinds of restaurants
2. **Yankee**: A parquet file that contains crowdsourced restaurant data

You also have one additional dataset called **Lookup Record** that contains the pre-defined mapping of restaurants between various rows of Delta through Yankee. Each row in the Lookup table can be considered as a single restaurant and can be assigned to one or more sources based on whether or not they exist for that restaurant. The Lookup records may have imperfect matches between sources, but you can ignore this quality issue and instead consider it as a source of truth for the amalgamation stage.


### License 📜

The data in this assignment is provided by Dotlas and is available for non-commercial use, such as research previews, and open-source projects. For any commercial use of the data, you must request access and provide all necessary context to [Dotlas](https://www.dotlas.com). Unauthorized commercial use of the data is prohibited. For more details, refer to the `DATA_LICENSE` file in the GitHub repository.

In [2]:
delta_url: str = (
    "https://dotlas-marketing.s3.amazonaws.com/interviews/california_delta_dataset.csv"
)
oscar_url: str = (
    "https://dotlas-marketing.s3.amazonaws.com/interviews/california_oscar_dataset.pkl"
)
tango_url: str = (
    "https://dotlas-marketing.s3.amazonaws.com/interviews/california_tango_dataset.json"
)
yankee_url: str = (
    "https://dotlas-marketing.s3.amazonaws.com/interviews/california_yankee_dataset.parquet"
)

lookup_record: str = (
    "https://dotlas-marketing.s3.amazonaws.com/interviews/california_master_record.csv"
)

grf4egrjbknhhnddsd    hnk# 1.1 Read Data 🔀 [2]

Load each source data file into separate DataFrames. Do not download the files locally to the file system but instead read them directly in-memory.

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def read_files_into_dataframes(file_path):
  df=pd.DataFrame()



  try:
            # Determine the file format based on the file extension
      if file_path.endswith('.csv'):
          df = pd.read_csv(file_path)
      elif file_path.endswith('.parquet'):
          df = pd.read_parquet(file_path)
      elif file_path.endswith('.pkl'):
          df = pd.read_pickle(file_path)
      elif file_path.endswith('.json'):
          df=pd.read_json(file_path)
      else:
          raise ValueError(f"Unsupported file format: {file_path}")

            # Store the DataFrame in the dictionary with the specified name


      print(f"Successfully read  ")

  except Exception as e:
            # Handle exceptions
      print(f"Error reading : {str(e)}")

  return df




# Access the DataFrames using their names

table_c = read_files_into_dataframes(yankee_url)
table_a = read_files_into_dataframes(oscar_url)
table_b = read_files_into_dataframes(tango_url)
table_d  = pd.read_csv(delta_url , delimiter='|')
table_e = read_files_into_dataframes(lookup_record)

Successfully read  
Successfully read  
Successfully read  
Successfully read  


## functions
1. replace_city\
    for table_c we have abnormality of address so we handled here
2. handling_address\
    for table_a we have abnomality of addree so we handled here
3. remove_html_tags\
    for removing html tags from text
4. handling_html_tags\
    using above function(removing_html_tags) to remove tags from columns
5. handling_space\
    convert space with None
6. has_html_tags\
    to  check wether our column have html tags
7. keep_unique_value\
    in our list only keep unique value after removing extra space or upper/lower case case
9. check_url\
    to check wether url in right format
10. check_url_change_none\
      put None to those urls which are not in right format
11. website_url_formatting\
      format proper to http or https and remove extra space
12. check_number\
      to check wether our number are in correct format of (ddd) ddd-dddd
13. fromat_phone_number\
      format our phone number to (ddd) ddd-dddd format
14. add_www\
      some website might be usable if we just add www. at starting (special case )
15. clean_and_format_facebook \
      to clean our facebook urls.
16. *convert_operation: \
      its special function for table_a operating_hours \
      where it convert our text to list according to monday-sunday \
      and put value on them in format of "9:00 AM to 5:00 PM" \
      contain multiple internal function for handling data \
      1.for extracting days \
      2.for extracting times \
      3.mapping days form 0-7 \

      it will only return if our both extracted day format and time match else it will return None
17. union_and_process_lists \
      it will merge two list and keep unique of them
18. find_columns_with_empty_values \
      to get us all rows which conatin space or if list column then tell if any list have none in it and tell if any our column contain None in it




In [4]:

def replace_city(row):
  if row['address']:
    if row['city'] in row['address']:
        return row['address'].replace(row['city'], f", {row['city']}")
  return row["address"]
def handling_address(x):
  if x is not None:
    x=x.replace(', ,','@').replace(' CA,','').replace(',',' , CA',1).replace("@"," , " )
  return x
def remove_html_tags(text):
    if text is None:
        return None
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()
def handling_html_tags(df,column):
  df[column]=df[column].apply(remove_html_tags)
  df[column]=df[column].apply(lambda value : None if value == '' else value)


def handling_space(df,column):
  df[column]=df[column].apply(lambda value : None if value == "" else value )

def handling_space_operation(df,column):
  df[column]=df[column].apply(lambda value : None if value == "" else value )


def has_html_tags(text):
    if text is None:
        return False

    # Regular expression to match HTML tags
    html_tag_pattern = re.compile(r'<.*?>')

    # Search for HTML tags in the text
    return bool(html_tag_pattern.search(text))

def keep_unique_values(strings):
    cleaned_strings = {}
    result = []

    for string in strings:
        # Clean the string by removing extra spaces and making it lowercase
        cleaned_string = ' '.join(string.split()).lower()

        # Check if the cleaned string is already in the dictionary
        if cleaned_string not in cleaned_strings:
            # If not, add it to the dictionary and the result list
            cleaned_strings[cleaned_string] = True
            result.append(string)

    return result

def check_url(df,column):
  filtered_urls=pd.DataFrame()
  url_pattern = r'^(https?://)[^\s/$.?#].[^\s]*$'
  filtered_urls = df[df[column].notna()]
  filtered_urls = filtered_urls[~filtered_urls[column].str.contains(url_pattern)]
  print(filtered_urls.shape)
  print(filtered_urls[column].head(2))

def check_url_change_none(df,column):
  filtered_urls=pd.DataFrame()
  url_pattern = r'^(https?://)[^\s/$.?#].[^\s]*$'
  filtered_urls = df[df[column].notna()]
  filtered_urls = filtered_urls[~filtered_urls[column].str.contains(url_pattern)]
  print(filtered_urls.shape)
  index=filtered_urls.index
  for i, index_value in enumerate(filtered_urls.index):
    df.at[index_value,column] = None


def website_url_fomatting(url):
    # Convert the URL to lowercase
    if url is None:
      return None
    url = url.lower()

    # Check if the URL starts with "http://" or "https://"
    if url.startswith("http://"):
        # Replace "http://" with "https://www."
        url = url.split("www.", 1)[-1]
        url = "https://www." + url[len("http://"):]
    elif url.startswith("https://"):
        # Replace "https://" with "https://www."
        url = url.split("www.", 1)[-1]
        url = "https://www." + url[len("https://"):]

    # Remove the specific substring "it will be available soon"

    return url

def check_number(df,column):
  pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
  all_in_desired_format = df[column].str.match(pattern, na=False).all()
  print(all_in_desired_format)
def format_phone_number(phone):
    if phone is None:
        return None
    phone=phone.replace("+1","")
    phone = ''.join(filter(str.isdigit, phone))  # Remove non-digit characters
    if len(phone) == 10:  # Check if it's a valid phone number
        return f'({phone[:3]}) {phone[3:6]}-{phone[6:]}'
    else:
        return None

def add_www(url):
    if url is not None and not url.startswith('www.') and not url.startswith('http'):
        return 'www.' + url
    else:
        return url
def clean_and_format_facebook(url):
    if url is not None:
        # Remove any leading/trailing spaces
        url = url.strip()

        # Use regular expression to extract the portion starting with 'https://'
        match = re.search(r'https://.*', url)
        if match:
            url = match.group(0)
        else:
            # If 'https://' is not found, consider the entire URL
            url = None  # or url = '' if you want to keep it as an empty string

    return url

def convert_operation(text):
  if text==None :
    return None

  def extract_days(text):
    # Replace "Friday" with "Fri"
    # Find all matches of day-day format
      text = re.sub(r'\bMonday\b', 'Mon', text, flags=re.IGNORECASE)
      text = re.sub(r'\bTuesday\b', 'Tue', text, flags=re.IGNORECASE)
      text = re.sub(r'\bWednesday\b', 'Wed', text, flags=re.IGNORECASE)
      text = re.sub(r'\bThursday\b', 'Thu', text, flags=re.IGNORECASE)
      text = re.sub(r'\bFriday\b', 'Fri', text, flags=re.IGNORECASE)
      text = re.sub(r'\bSaturday\b', 'Sat', text, flags=re.IGNORECASE)
      text = re.sub(r'\bSunday\b', 'Sun', text, flags=re.IGNORECASE)


      pattern = r'\b(?:Mon(?:day)?|Tue(?:sday)?|Wed(?:nesday)?|Thu(?:rsday)?|Fri(?:day)?|Sat(?:urday)?|Sun(?:day)?)\s*(?:-|–|to)\s*(?:Mon(?:day)?|Tue(?:sday)?|Wed(?:nesday)?|Thu(?:rsday)?|Fri(?:day)?|Sat(?:urday)?|Sun(?:day)?)\b|\b(?:Mon(?:day)?|Tue(?:sday)?|Wed(?:nesday)?|Thu(?:rsday)?|Fri(?:day)?|Sat(?:urday)?|Sun(?:day)?)\b'
      matches = re.findall(pattern, text, flags=re.IGNORECASE)

      return matches
  def split_list(input_list):

      re=[]
      for list in range(0,len(input_list),2):
        if list + 1 < len(input_list):
          first=input_list[list].replace(" ","")
          second=input_list[list+1].replace(" ","")
          final=first+" to "+second
          re.append(final)
    # Check for remaining elements
  def extract_time_patterns(operating_hours):
    # Define a regular expression to match time patterns
      time_pattern = r'\d+:\d+\s*[apm]+'

    # Use regular expression to extract time patterns
      time_patterns = re.findall(time_pattern, operating_hours)
      time=split_list(time_patterns)


      return time
  days=extract_days(text)
  time=extract_time_patterns(text)
  def mapping(days,time):
      days_mapping = {
      'Mon': 0,
      'Tue': 1,
      'Wed': 2,
      'Thu': 3,
      'Fri': 4,
      'Sat': 5,
      'Sun': 6
      }
      result = [[] for _ in range(7)]
      if len(days)==0:
          return None
      len_t=len(time)

      for i in range(0,len(days)):
          if i>=len_t:
              return None

          days[i] = ''.join([c if unicodedata.category(c) != 'Pd' else '-' for c in days[i]])
          days[i]=days[i].replace("to","-")

          if '-'  in days[i]:
              start,end= days[i].split("-")
              start=start.replace(" ","")
              end=end.replace(" ","")
              start_i=days_mapping[start]
              end_i=days_mapping[end]
              for j in range(start_i,end_i+1):
                  result[j].append(time[i])
          else:
              start=days[i].replace(" ","")
              start_i=days_mapping[start]
              result[start_i].append(time[i])

      return result
  def format_time(time_str):
    if time_str:
        # Capitalize 'am' and 'pm', add space before AM/PM
        formatted_time = time_str.replace('am', 'AM').replace('pm', 'PM').replace('AM', ' AM').replace('PM', ' PM')
        return formatted_time
    return None
  if days ==None or time==None:
    return None
  if len(days)!= len(time) :
      return None

  final_result=mapping(days,time)
  if final_result ==None:
    return None
  result=[" , ".join(map(str,sublist)) if sublist else None for sublist in final_result]
  finall=[]
  for i in result:
    v=format_time(i)
    finall.append(v)
  return finall


def union_and_process_lists(list1, list2):
    if isinstance(list1, list) and isinstance(list2, list):
        # Combine the lists without modifying the original values
        list1=[item.strip().lower().replace("'","") for sublist in list1 for item in sublist.split(',')]
        list2=[item.strip().lower().replace("'","") for sublist in list2 for item in sublist.split(',')]

        union_lower = list(set(list1 + list2))
        result_list = [value.strip().title() for value in union_lower]
        return result_list
    elif isinstance(list1, list):

        return list1  # list1 is a list, return list1
    elif isinstance(list2, list):
        return list2  # list2 is a list, return list2

    else:
        return None


def find_columns_with_empty_values(df):
    empty_columns = []

    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, str) and x == "").any():
            empty_columns.append((column," have space"))
        elif df[column].apply(lambda x: isinstance(x, list) and len(x) == 0).any():
            empty_columns.append((column,"empty list"))
        elif df[column].apply(lambda x: x is None).any():
            empty_columns.append((column,"have none"))

    return empty_columns


### 1.2 Exploratory Data Analysis 📊 [3]

Perform a simple exploratory data analysis of each dataset to understand its structure, null values, nature of the data, data types, duplicate analysis, data quality and more.

 ### seeking information about table_a
 \ accesing data type in our dataframe
  checking for if duplicate of brand_name and latitude and longitude \ present\
 checking for area,address,city,categories,description for pattern\
 checking how many have delivery partners\
 checking for phone_number\
 checking for facebook\


In [19]:
table_a.info()
print(find_columns_with_empty_values(table_a))
result_df = table_a[table_a.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]
display(result_df.shape)
display(table_a[["area","address","city"]].head(3))
display(table_a["categories"].head(3))
display(table_a["description"].head(3))
display(table_a[~table_a["has_delivery_partners"]].head(2))
display(table_a["phone_number"].head(3))
display(table_a["order_online_link"].head(3))
display(table_a["facebook"].head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3704 entries, 0 to 3703
Data columns (total 78 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   subregion                             3679 non-null   object 
 1   city                                  3704 non-null   object 
 2   brand_name                            3704 non-null   object 
 3   categories                            3704 non-null   object 
 4   latitude                              3704 non-null   float64
 5   longitude                             3704 non-null   float64
 6   area                                  3704 non-null   object 
 7   address                               3704 non-null   object 
 8   description                           3704 non-null   object 
 9   public_transit                        1063 non-null   object 
 10  cross_street                          1896 non-null   object 
 11  restaurant_websit

(0, 78)

,area,address,city
0,Danville,"120 E Prospect Avenue, , CA, Danville, 94526, ...",Danville
1,The Castro,"3583 16th Street, , CA, San Francisco, 94114, ...",San Francisco
2,Glen Park,"2788 Diamond Street, , CA, San Francisco, 9413...",San Francisco


0        [Greek, Mediterranean]
1    [Unspecified, Californian]
2                     [Italian]
Name: categories, dtype: object

0    <p>Yanni's Taverna is a Mediterranean restaura...
1    Starbelly is a modern, urban restaurant that s...
2    You don't have to fly to Italy for authentic I...
Name: description, dtype: object

,subregion,city,brand_name,categories,latitude,longitude,area,address,description,public_transit,cross_street,restaurant_website,phone_number,primary_cuisine,dining_style,executive_chef_name,parking_info,dress_code,entertainment,operating_hours,price_range_id,price_range,payment_options,maximum_days_advance_for_reservation,rating,rating_count,atmosphere_rating,noise_rating,food_rating,service_rating,value_rating,terrible_review_count,poor_review_count,average_review_count,very_good_review_count,excellent_review_count,most_recent_review,review_count,review_topics,tags,has_clean_menus,has_common_area_cleaning,has_common_area_distancing,has_contact_tracing_collected,has_contactless_payment,requires_diner_temperature_check,has_limited_seating,prohibits_sick_staff,has_proof_of_vaccination_outdoor,requires_proof_of_vaccination,requires_diner_masks,requires_wait_staff_masks,has_sanitized_surfaces,provides_sanitizer_for_customers,has_sealed_utensils,has_vaccinated_staff,requires_staff_temp_checks,has_table_layout_with_extra_space,is_permanently_closed,is_waitlist_only,has_waitlist,has_bar,has_counter,has_high_top_seating,has_outdoor_seating,has_priority_seating,has_private_dining,has_takeout,has_delivery_partners,has_pickup,is_network_non_bookable,has_gifting,order_online_link,delivery_partners,facebook,menu_url,daily_reservation_count,id
0,California,Danville,Yanni's Taverna,"[Greek, Mediterranean]",37.82196,-121.99948,Danville,"120 E Prospect Avenue, , CA, Danville, 94526, ...",<p>Yanni's Taverna is a Mediterranean restaura...,None,Hartz Avenue,http://www.yannistaverna.com/,(925) 820-7700,Greek,Casual Dining,None,Street Parking,Casual Dress,Live Greek music once every month.,Lunch: Monday - Saturday: 11:00am - 4:00pm Din...,2,$30 and under,"[AMEX, Discover, Mastercard, Visa]",90,4.1,14,3.3,2.0,4.1,4.1,3.6,0,0,4,5,5,Great find! Mixed grill is fabulous!,7,"[Kid-friendly, Innovative, Good for groups]","[Bar Dining, Beer, Corkage Fee, Counter Seatin...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,None,[],None,http://www.yannistaverna.com/#!menus/cfvg,NaN,69343cdc34bdb16230c1cb125cf64a6d53420c30ba101f...
1,California,San Francisco,Starbelly,"[Unspecified, Californian]",37.76408,-122.43257,The Castro,"3583 16th Street, , CA, San Francisco, 94114, ...","Starbelly is a modern, urban restaurant that s...","Castro Metro Stop, 33 & 37 bus line.",Market Street,http://www.starbellysf.com/,(415) 252-7500,Unspecified,Casual Dining,Adam Timney,Street Parking,Casual Dress,None,Mon–Thu 11:00 am–9:00 pm Fri 11:00 am–10:00 pm...,2,$30 and under,"[AMEX, Discover, Mastercard, Visa]",90,4.6,9375,4.4,2.0,4.5,4.4,4.3,79,190,654,1936,6516,We did not get a quality check after getting o...,1421,"[Neighborhood gem, Great for outdoor dining, R...","[Banquet, Beer, BYO Wine, Cocktails, Full Bar,...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,None,[],http://www.facebook.com/StarbellySF,http://starbellysf.com/#section-menu,NaN,278f1579ec003d836f60e660a77b6045138e88a7b2cfbc...


0    (925) 820-7700
1    (415) 252-7500
2    (415) 334-2251
Name: phone_number, dtype: object

0    None
1    None
2    None
Name: order_online_link, dtype: object

0                                                 None
1                  http://www.facebook.com/StarbellySF
2    http://www.facebook.com/pages/Manzoni/11153140...
Name: facebook, dtype: object

## checking for table_b
checking for column if they contain None or space or list containing any None value\
checking for value area,address,city,categories,meals_offered,description,rating\
checking for different timezone and seeing city address for them
checking for valid email_address and getting how many are not

In [20]:
table_b.info()
print(find_columns_with_empty_values(table_b))
result_df = table_a[table_b.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]
display(result_df.shape)

display(table_b[["area","address","city"]].head(3))
display(table_b["categories"].head(3))
display(table_b["meals_offered"].head(3))
display(table_b["description"].head(3))
display(table_b["rating"].value_counts())

table_b_different_timezone=table_b[~(table_b["operating_timezone"]=="America/Los_Angeles")]
table_b_different_timezone=table_b_different_timezone[table_b_different_timezone["operating_timezone"].notna()]
display(table_b_different_timezone[["operating_timezone","city","address"]])

table_b['is_valid_email'] = table_b['email_address'].str.contains(r'^(?=.*@)(?=.*\.)', na=True)
value=table_b["email_address"].value_counts()
sum=value.sum()
print(sum)
print(table_b["is_valid_email"].value_counts())
invalid_email_rows = table_b['is_valid_email'] == False
display(invalid_email_rows.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   industry                25485 non-null  object 
 1   subregion               25485 non-null  object 
 2   city                    25485 non-null  object 
 3   brand_name              25485 non-null  object 
 4   categories              25485 non-null  object 
 5   latitude                25485 non-null  float64
 6   longitude               25485 non-null  float64
 7   area                    0 non-null      float64
 8   address                 25485 non-null  object 
 9   email_address           5156 non-null   object 
 10  telephone_number        19827 non-null  object 
 11  restaurant_website      18007 non-null  object 
 12  rating                  25485 non-null  float64
 13  review_count            25485 non-null  int64  
 14  ranking                 16953 non-null

<ipython-input-20-4ac3fcd6ee75>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_df = table_a[table_b.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]


(10, 78)

,area,address,city
0,NaN,"4097 19th Ave, San Francisco, CA 94132-3054",San Francisco
1,NaN,"564 S Van Ness Ave, San Francisco, CA 94110-1303",San Francisco
2,NaN,"101 Horne Avenue, San Francisco, CA 94124",San Francisco


0    []
1    []
2    []
Name: categories, dtype: object

...

0    None
1    None
2    None
Name: description, dtype: object

-1.0    8529
 4.0    6062
 4.5    4987
 3.5    2284
 5.0    1854
 3.0    1166
 2.5     278
 2.0     191
 1.0      97
 1.5      37
Name: rating, dtype: int64

,operating_timezone,city,address
6722,America/Chicago,Columbia,"101 N James M Campbell Blvd Suite 14, Columbia..."
20686,America/Chicago,Pleasanton,"1401 2nd Street, Pleasanton, TX 78064"
21912,America/New_York,Waltham,"475 Winter Street, Waltham, MA 02451"
24698,America/Chicago,Campbell,"Highway 9 Opposite 5th Street, Campbell, MN 56522"


5156
True     25479
False        6
Name: is_valid_email, dtype: int64


(25485,)

## table_c checking
getting basic info and data type of table
checking for duplicated brandname,latitude and longitude
getting info on area,address,city,categories,operating_hours
checking for phone_number
looking for review_count and review_count_not_recommended
looking for pattern in is_claimed and is_claimable
looking for review_count_by_rating

In [21]:

table_c.info()
print(find_columns_with_empty_values(table_c))
result_df = table_a[table_c.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]
display(result_df.shape)
display(table_c[["area","address","city"]].head(3))
display(table_c["categories"].head(3))
display(table_c["operating_hours_mon"].value_counts())
phone_pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
invalid_numbers = table_c["telephone_number"].str.match(phone_pattern, na=None)
print(invalid_numbers.value_counts())
display(table_c[["review_count","review_count_not_recommended"]])
display(table_c[["is_claimed","is_claimable"]].value_counts())
display(table_c["review_count_by_rating"].head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33372 entries, 0 to 33371
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   subregion                      33372 non-null  object 
 1   city                           33372 non-null  object 
 2   brand_name                     33372 non-null  object 
 3   categories                     33372 non-null  object 
 4   latitude                       33372 non-null  float64
 5   longitude                      33372 non-null  float64
 6   area                           8565 non-null   object 
 7   address                        33372 non-null  object 
 8   cross_streets                  7114 non-null   object 
 9   restaurant_provided_menu_link  11356 non-null  object 
 10  website_url                    22299 non-null  object 
 11  is_claimable                   33372 non-null  bool   
 12  is_claimed                     33372 non-null 

<ipython-input-21-58b6ca1f204b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_df = table_a[table_c.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]


(4, 78)

,area,address,city
0,None,"1104 W Tennyson Rd Hayward, CA 94544",Hayward
1,None,"390 W Tennyson Rd Hayward, CA 94544",Hayward
2,None,"Hayward, CA 94544",Hayward


0                         [Mexican]
1    [Fast Food, Hawaiian, Chinese]
2                        [Cupcakes]
Name: categories, dtype: object

                                          7374
Closed                                    4154
11:00 AM - 9:00 PM                        1412
11:00 AM - 10:00 PM                        754
Open 24 hours                              697
                                          ... 
11:00 AM - 2:00 PM, 5:30 PM - 7:30 PM        1
12:00 PM - 3:00 PM, 5:00 PM - 8:00 PM        1
9:00 AM - 9:30 AM                            1
10:00 AM - 6:45 PM                           1
6:30 AM - 10:30 AM, 12:00 PM - 2:00 PM       1
Name: operating_hours_mon, Length: 1275, dtype: int64

True     28801
False     4571
Name: telephone_number, dtype: int64


,review_count,review_count_not_recommended
0,115.0,9.0
1,5.0,0.0
2,0.0,0.0
3,2.0,0.0
4,7.0,2.0
...,...,...
33367,1439.0,128.0
33368,2021.0,119.0
33369,12.0,3.0
33370,449.0,38.0


is_claimed  is_claimable
True        False           13708
False       True            11268
True        True             8356
False       False              40
dtype: int64

0    [26, 4, 14, 35, 36]
1        [1, 2, 0, 0, 2]
2        [0, 0, 0, 0, 0]
Name: review_count_by_rating, dtype: object

## checking table_d
getting data type of table\
finding column with empty spaces,none value,and list with none\
looking for area,address and city\
categories


In [22]:


table_d.info()

print(find_columns_with_empty_values(table_d))

result_df = table_a[table_d.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]
display(result_df.shape)

display(table_d[["area","address","city"]].head(3))
display(table_d["categories"].head(3))




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17845 entries, 0 to 17844
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   industry                17845 non-null  object 
 1   subregion               17845 non-null  object 
 2   city                    17844 non-null  object 
 3   brand_name              17845 non-null  object 
 4   categories              17845 non-null  object 
 5   latitude                17845 non-null  float64
 6   longitude               17845 non-null  float64
 7   area                    17797 non-null  object 
 8   address                 17845 non-null  object 
 9   brand_description       17786 non-null  object 
 10  rating                  17845 non-null  float64
 11  rating_count            14333 non-null  float64
 12  price_range             17623 non-null  object 
 13  operating_hours         17824 non-null  object 
 14  offers_delivery         17845 non-null

<ipython-input-22-dd86f4dd3b7c>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_df = table_a[table_d.duplicated(subset=['brand_name', 'latitude','longitude'], keep=False)]


(17, 78)

,area,address,city
0,146 Sunset Drive,"146 Sunset Dr, San Ramon, CA 94583, USA",San Ramon
1,1800 Fillmore Street,"1800 Fillmore St, San Francisco, CA 94115, USA",San Francisco
2,NaN,"STIX Eatery, 1500 Monument Blvd #A-2a, Concord...",Concord


0    ['Healthy', 'Salads', 'Vegan', 'Vegetarian']
1                                     ['Chicken']
2                                       ['Asian']
Name: categories, dtype: object

In [23]:

table_a= table_a.applymap(lambda x: None if isinstance(x, str) and (x == '' or x.isspace()) else x)
table_b=table_b.applymap(lambda x: None if isinstance(x, str) and (x == '' or x.isspace()) else x)
table_c = table_c.applymap(lambda x: None if isinstance(x, str) and (x == '' or x.isspace()) else x)
table_d = table_d.applymap(lambda x: None if isinstance(x, str) and (x == '' or x.isspace()) else x)

## cleaning up table_a a.k.a "oscar"
1.removing buplicate if both address and brand_name are same
2.removing duplicates if brand_name and latitude and longitude are same
3.filtering our area column getting data from address first phrase before ","\
4.handling our address removing abnormality for them as we have noticed like removing ", ," and putting CA in right place as formatted
5.correcting our has_delivery_partners ,if we have length of deliver_partner then True
6.correcting our phone_number in correct format of (ddd) ddd-dddd
7.removing html tags from entertainment column and also removing N/A if present
8.handling our order_online_link adding www where required
9.formating our facebook-link
10.handling our menu_url with check_url_and_none function
 ###* parsing our raw data in operating_column and getting data according to it and putting in 7 different column (as mon,tue........)

In [7]:
table_a= table_a.drop_duplicates(subset=['brand_name', 'address'], keep='first')

table_a = table_a.drop_duplicates(subset=['brand_name', 'latitude','longitude'], keep='first')

table_a=table_a[~table_a["is_permanently_closed"]]

handling_html_tags(table_a,"description")

table_a["categories"]=table_a["categories"].apply(keep_unique_values)
table_a['area_1'] = table_a['address'].str.split(',').str[0]
table_a = table_a.drop('area', axis=1)
# Rename 'area_1' to 'area'
table_a = table_a.rename(columns={'area_1': 'area'})
table_a["address"]=table_a["address"].apply(handling_address)

table_a["has_delivery_partners"]=table_a["delivery_partners"].apply(lambda value:True if (value is not None and len(value)>0) else False)

check_number(table_a,"phone_number")
table_a["phone_number"]=table_a["phone_number"].apply(format_phone_number)
phone_pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
table_a["phone_number"] = table_a["phone_number"].str.strip()  # Strip leading and trailing spaces
table_a["phone_number"]=table_a["phone_number"].apply(format_phone_number)
invalid_numbers = table_a["phone_number"].str.match(phone_pattern, na=None).all()
print(invalid_numbers)

table_a["entertainment"] = table_a["entertainment"].apply(remove_html_tags)
table_a["entertainment"]=table_a["entertainment"].apply(lambda value : None if value == ''or value =="N/A" else value)

table_a["order_online_link"]=table_a["order_online_link"].apply(add_www)

table_a["facebook"]=table_a["facebook"].apply(clean_and_format_facebook)

check_url(table_a,"menu_url")
check_url_change_none(table_a,"menu_url")

column_names = ['mon', 'tue', 'wed', 'thu','fri','sat','sun']

# Create an empty DataFrame with the specified column names
df = pd.DataFrame(columns=column_names)

handling_space(table_a,"operating_hours")

table_a["operating_hours"]=table_a["operating_hours"].apply(convert_operation)


for day, col_name in zip([0, 1, 2, 3, 4, 5, 6],
                        [f'operating_hours_{day.lower()}' for day in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']]):
    table_a[col_name] = table_a['operating_hours'].apply(lambda x: (x[day] if x and day<len(x) else None))


<ipython-input-4-af122354b1f4>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


False
True
(2, 78)
163        http://See Banquet Menu on Website
943    http://Instagram: @joraperuviancuisine
Name: menu_url, dtype: object
(2, 78)


<ipython-input-4-af122354b1f4>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
<ipython-input-4-af122354b1f4>:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_urls = filtered_urls[~filtered_urls[column].str.contains(url_pattern)]
<ipython-input-4-af122354b1f4>:65: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_urls = filtered_urls[~filtered_urls[column].str.contains(url_pattern)]


## cleaning up table_b a.k.a "tango"
1.dropping duplicate if both brand_name and address are same\
2. droping duplicates if our brand_name and latitude and longitude are same\
3. handling our area ,getting from address column, first phrase before ","
4. from table_b removed which are closed
5. sorting our meals_offered column so list having same data in different index are treated same
6.handling our invalid_email getting how many are different and putting none if not
7.formatting our phone_number
8.making our rating column free from negative number and having absolute value (i have not put them as 0 if negative)
9.looking for different_time_zone and looking if they match with address and city

In [8]:
table_b= table_b.drop_duplicates(subset=['brand_name', 'address'], keep='first')

table_b = table_b.drop_duplicates(subset=['brand_name', 'latitude','longitude'], keep='first')

table_b=table_b[~table_b["is_closed"]]

table_b['area_1'] = table_b['address'].str.split(',').str[0]
table_b = table_b.drop('area', axis=1)
table_b = table_b.rename(columns={'area_1': 'area'})
table_b = table_b.where(pd.notna(table_b), None)

table_b["meals_offered"]=table_b["meals_offered"].apply(lambda lst: sorted(lst) if lst else None)

table_b['is_valid_email'] = table_b['email_address'].str.contains(r'^(?=.*@)(?=.*\.)', na=True)
value=table_b["email_address"].value_counts()
sum=value.sum()
print(sum)
print(table_b["is_valid_email"].value_counts())
invalid_email_rows = table_b['is_valid_email'] == False
display(invalid_email_rows.shape)
table_b.loc[invalid_email_rows, 'email_address'] = None

table_b["telephone_number"]=table_b["telephone_number"].apply(format_phone_number)

handling_html_tags(table_b,"description")

table_b["rating"]=table_b["rating"].apply(lambda value: abs(value))

table_b_different_timezone=table_b[~(table_b["operating_timezone"]=="America/Los_Angeles")]
table_b_different_timezone=table_b_different_timezone[table_b_different_timezone["operating_timezone"].notna()]

5154
True     25453
False        6
Name: is_valid_email, dtype: int64


(25459,)

<ipython-input-4-af122354b1f4>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


## cleaning up table_c a.k.a "yankee"

1.dropping duplicate if both brand_name and address are same\
2. droping duplicates if our brand_name and latitude and longitude are same\
3. putting data in our area column from address, first phrase before ","
4. checking our operating_hours_xxx for closed first finding in which columns we have to perform and then removing Closed from them
5.handling address , adding city in them
6.formatting our phone number getting how many are not compatible with our pattern
7.handling our address column replacing city at its right place
8.handling review_count ,subtracting review_count from review_not_recommended handling negative number with zero\
9.making our is_claimed right by comparing with is_claimable and putting value if is_claimed is false and and is_claimable is true
10handling year established if we have year below 1785 removing them
11.handling review_count_by_rating into five column normalizing as ["terrible,poor,average,very_good,excellent]


In [27]:
table_c= table_c.drop_duplicates(subset=['brand_name', 'address'], keep='first')

table_c = table_c.drop_duplicates(subset=['brand_name', 'latitude','longitude'], keep='first')

# table_c = table_c.where(pd.notna(table_c), None)

table_c['area_1'] = table_c['address'].str.split(',').str[0]
table_c = table_c.drop('area', axis=1)
table_c = table_c.rename(columns={'area_1': 'area'})


columns_to_check = ['operating_hours_mon', 'operating_hours_tue', 'operating_hours_wed', 'operating_hours_thu', 'operating_hours_fri', 'operating_hours_sat', 'operating_hours_sun']

# Check and replace "Closed" with None in the specified columns
table_c[columns_to_check] = table_c[columns_to_check].applymap(lambda x: None if isinstance(x, str) and "Closed" in x else x)
table_c = table_c.applymap(lambda x: None if isinstance(x, str) and "Closed" in x else x)


table_c["address"]=table_c.apply(replace_city,axis=1)

check_number(table_c,"telephone_number")
table_c["telephone_number"]=table_c["telephone_number"].apply(format_phone_number)
phone_pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
table_c["telephone_number"] = table_c["telephone_number"].str.strip()  # Strip leading and trailing spaces
table_c["telephone_number"]=table_c["telephone_number"].apply(format_phone_number)
invalid_numbers = table_c["telephone_number"].str.match(phone_pattern, na=None)
print(invalid_numbers.value_counts())

table_c['review_count'] = table_c['review_count'] - table_c['review_count_not_recommended']
table_c['review_count'] = table_c['review_count'].apply(lambda x: max(x, 0))

table_c['is_claimed'] = table_c.apply(lambda row: False if row['is_claimable'] == True and row['is_claimed'] == False else row['is_claimed'], axis=1)
# table_c=table_c.drop('is_claimable', axis=1, inplace=True)
table_c["year_established"]=table_c["year_established"].apply(lambda value : None if (value == None or int(value)<1785) else str(value)[0:4])

table_c["review_check"]=table_c["review_count_by_rating"].apply(lambda lst: True if  lst is not None and len(lst)==5 else False)


table_c = table_c.join(table_c['review_count_by_rating'].apply(pd.Series).rename(columns=lambda x: f'new_column_{x}'))
column_mapping = {
    'new_column_0': 'terrible_review_count',
    'new_column_1': 'poor_review_count',
    'new_column_2': 'average_review_count',
    'new_column_3': 'very_good_review_count',
    'new_column_4': 'excellent_review_count'
}


table_c.rename(columns=column_mapping, inplace=True)

TypeError: ignored

In [5]:
table_c["address"]=table_c.apply(replace_city,axis=1)

columns_to_check = ['operating_hours_mon', 'operating_hours_tue', 'operating_hours_wed', 'operating_hours_thu', 'operating_hours_fri', 'operating_hours_sat', 'operating_hours_sun']


table_c['area_1'] = table_c['address'].str.split(',').str[0]
table_c = table_c.drop('area', axis=1)
table_c = table_c.rename(columns={'area_1': 'area'})
# Check and replace "Closed" with None in the specified columns
table_c[columns_to_check] = table_c[columns_to_check].applymap(lambda x: None if isinstance(x, str) and "Closed" in x else x)
table_c = table_c.applymap(lambda x: None if isinstance(x, str) and "Closed" in x else x)

table_c["telephone_number"]=table_c["telephone_number"].apply(format_phone_number)
phone_pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
table_c["telephone_number"] = table_c["telephone_number"].str.strip()  # Strip leading and trailing spaces
table_c["telephone_number"]=table_c["telephone_number"].apply(format_phone_number)
invalid_numbers = table_c["telephone_number"].str.match(phone_pattern, na=None)
print(invalid_numbers.value_counts())

handling_space(table_c,"specialties")

handling_space(table_c,"year_established")
table_c["year_established"]=table_c["year_established"].apply(lambda value : None if (value == None or int(value)<1785) else str(value)[0:4])


table_c["review_check"]=table_c["review_count_by_rating"].apply(lambda lst: True if len(lst)==5 else False)
print(table_c["review_check"].value_counts())
table_c=table_c.drop('review_check', axis=1)

table_c = table_c.join(table_c['review_count_by_rating'].apply(pd.Series).rename(columns=lambda x: f'new_column_{x}'))
column_mapping = {
    'new_column_0': 'terrible_review_count',
    'new_column_1': 'poor_review_count',
    'new_column_2': 'average_review_count',
    'new_column_3': 'very_good_review_count',
    'new_column_4': 'excellent_review_count'
}


table_c.rename(columns=column_mapping, inplace=True)

True    28802
Name: telephone_number, dtype: int64
True    33372
Name: review_check, dtype: int64


## cleaning up table_d a.k.a "delta"
converting na as none
removing duplicates if brand_name and latitude and longitude are same
removing duplicates if brand_name and address are same
cleaning our categories column removing unwanted character from them

In [6]:
table_d['area'] = table_d['area'].astype(object)
table_d = table_d.where(pd.notna(table_d), None)

def clean_category_list(category_str):
    # Remove empty strings, extra spaces, and single quotes from each element
    if category_str == None:
      return None
    category_list = ast.literal_eval(category_str)
    cleaned_list = [category.strip().replace("'", "") for category in category_list if category.strip()]
    return cleaned_list

# Apply the clean_category_list function to the 'categories' column
table_d= table_d.drop_duplicates(subset=['brand_name', 'address'], keep='first')
table_d= table_d.drop_duplicates(subset=['brand_name', 'latitude','longitude'], keep='first')
table_d['categories'] = table_d['categories'].apply(clean_category_list)

## Section 2: Join and Knit 🧵 [15]

<img src="https://media.giphy.com/media/rytLWOErAX1F6/giphy.gif" width="350px" alt="merge">
<br></br>

1. **Objective**:
   - Merge the datasets - Delta, Oscar, Tango, and Yankee into one final restaurant dataset.
   - Use the *Lookup Records* table for reference on joining the source tables.

2. **Key Details**:
   - Datasets contain unique details (e.g., Dining Style, Meals Offered).
   - Some details (e.g., Name, Location) are shared across datasets.
   - Your goal: Merge to create `Table Z`.

3. **How to Merge**:
   - Join tables Delta through Yankee using **all columns**.
   - Address overlapping columns to avoid duplication.
   - Merge columns with similar data but different names (like 'telephone_number' and 'phone_number').
   - If a column appears in multiple tables with varying values, select the most appropriate one (e.g., 'price_range' and 'price_class').

### Example

Consider the hypothetical tables A, B, C, and D. We aim to merge these into Table Z.

Note: The example highlights specific columns from A to D, but you must consider all columns from datasets Delta to Yankee.

**Table A**

| id | restaurant_name | area | categories | rating | website | price_range | executive_chef |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | The Good Food | Downtown | Italian,Seafood | 4.5 | goodfood.com | $11-30 | John Doe |
| 2 | Taste of Asia | Midtown | Asian,Thai | 4.0 | tasteofasia.com | $10-25 | Jane Doe |

**Table B**

| id | name | location | type | rating | website | price_class | meals_offered |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | The Good Food | Downtown | Italian | 4.5 | goodfood.com | 2 | Lunch,Dinner |
| 3 | Spicy Hut | Uptown | Indian,Spicy | 4.2 | spicyhut.com | 3 | Lunch,Dinner,Brunch |

**Table C**

| id | restaurant | location | categories | telephone_number | price_range |
| --- | --- | --- | --- | --- | --- |
| 2 | Taste of Asia | Midtown | Asian,Thai | 1234567890 | $10-25 |
| 4 | Fresh & Healthy | Suburbs | Vegetarian,Vegan | 9876543210 | $15-30 |

**Table D**

| id | restaurant_name | area | type | phone_number | website | executive_chef |
| --- | --- | --- | --- | --- | --- | --- |
| 3 | Spicy Hut | Uptown | Indian | 1234567890 | spicyhut.com | Raj Patel |
| 4 | Fresh & Healthy | Suburbs | Vegetarian | 9876543210 | freshandhealthy.com | Emily Brown |

We want to merge tables A, B, C, and D into a single table Z. However, we need to handle the overlapping and exclusive columns carefully to avoid duplication and to ensure a single source of truth. We will use coalescing to handle overlapping columns with different names (e.g., 'telephone_number' and 'phone_number') and combine values when necessary (e.g., 'categories' and 'type'). We will also choose the most appropriate value for columns that appear in multiple tables but have different values (e.g., 'price_range' and 'price_class').

**Table Z**

| id | name | location | categories | rating | website | price_range | executive_chef | meals_offered | telephone_number |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | The Good Food | Downtown | Italian,Seafood | 4.5 | goodfood.com | $11-30 | John Doe | Lunch,Dinner | |
| 2 | Taste of Asia | Midtown | Asian,Thai | 4.0 | tasteofasia.com | $10-25 | Jane Doe | | 1234567890 |
| 3 | Spicy Hut | Uptown | Indian,Spicy | 4.2 | spicyhut.com | $25-40 | Raj Patel | Lunch,Dinner,Brunch | 1234567890 |
| 4 | Fresh & Healthy | Suburbs | Vegetarian,Vegan | | freshandhealthy.com | $15-30 | Emily Brown | | 9876543210 |


> 📝 It's important to note that while this example is straightforward, you may encounter some ambiguity when working with the actual datasets Delta through Yankee. **The main objective is to construct a dataset that provides a comprehensive and detailed context for each restaurant**. There will be instances where you'll need to make subjective decisions, such as prioritizing one dataset over another or creating a new field to harmonize different representations of the same data across datasets (e.g., your own classification of price range). These decisions are up to you, and we encourage you to develop your own approach and rationale for resolving these issues. Just make sure to explain your choices and reasoning in a markdown cell.

---

### Tips 💡
- Remember to clean up columns in the 4 source datasets where necessary that would make the join more seamless, instead of retrofitting ad-hoc fixes after the fact.
- If you have two or more columns with same or similar names or two or more different columns that are representing the same kind of data - then you have not reached the final output
- All values in a single column should have the same data type. A type can be simple like `integer`, `float`, `string`, or complex like `list[str]` or `list[int]`. Try to avoid columns that result in having `json` or `dict` values.
- The underlying four datasets and lookup records may have duplicates, so watch out for bugs in the join

### first merging on yankee and tango

In [9]:
#left merge on table_c on table_c and then on table_b
merged_df = table_e.merge(table_c, left_on='yankee', right_on='id', how='left'). \
    merge(table_b, left_on='tango', right_on='id', how='left',suffixes=("_yan","_tan"))

In [10]:
# looking for same latitude and longitude and filtering out if
filtered_df = merged_df[(merged_df['latitude_yan'] == merged_df['latitude_tan']) & (merged_df['longitude_yan'] == merged_df['longitude_tan'])]
display(filtered_df.head(3))
display(filtered_df.shape)
filtered_df = merged_df[(merged_df['id_yan'].notna()) & (merged_df['id_tan'].notna())]
filtered_df = filtered_df[filtered_df['id_yan'].isin(filtered_df['id_tan'])]
display(filtered_df.shape)

,oscar,tango,yankee,delta,subregion_yan,city_yan,brand_name_yan,categories_yan,latitude_yan,longitude_yan,address_yan,cross_streets,restaurant_provided_menu_link,website_url,is_claimable,is_claimed,claimability_reason,operating_hours_mon_yan,operating_hours_tue_yan,operating_hours_wed_yan,operating_hours_thu_yan,operating_hours_fri_yan,operating_hours_sat_yan,operating_hours_sun_yan,observed_holidays,place_features_yan,review_count_yan,rating_yan,review_count_not_recommended,telephone_number_yan,description_yan,specialties,year_established,price_range_yan,review_count_by_rating,id_yan,area_yan,terrible_review_count_yan,poor_review_count_yan,average_review_count_yan,very_good_review_count_yan,excellent_review_count_yan,industry,subregion_tan,city_tan,brand_name_tan,categories_tan,latitude_tan,longitude_tan,address_tan,email_address,telephone_number_tan,restaurant_website,rating_tan,review_count_tan,ranking,awards,terrible_review_count_tan,poor_review_count_tan,average_review_count_tan,very_good_review_count_tan,excellent_review_count_tan,food_rating,service_rating,value_rating,atmosphere_rating,claimed,is_closed,description_tan,price_range_tan,operating_timezone,operating_hours_sun_tan,operating_hours_mon_tan,operating_hours_tue_tan,operating_hours_wed_tan,operating_hours_thu_tan,operating_hours_fri_tan,operating_hours_sat_tan,dietary_restrictions,meals_offered,place_features_tan,place_type_by_price,place_type_by_service,id_tan,area_tan,is_valid_email
0,59a9eb31e516040b310e64eb7ec071e4a6e8393ec674ab...,dfae318ebb270c4bbf79d9ec8e79a28e7943bc2e17f7e4...,eba22f6c961d84fc919bf8a62f47cc52ac1ad534c824e5...,3c528979d1d4bed643e90aee85130ef54f00d1c66b0bf5...,CA,Pittsburg,Outback Steakhouse,[Steakhouses],38.00588,-121.84010,"4350 Century Blvd , Pittsburg, CA 94565",None,https://www.outback.com/menu,https://locations.outback.com/california/pitts...,False,True,ALREADY_CLAIMED,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 11:00 PM,11:30 AM - 11:00 PM,11:30 AM - 9:00 PM,"[Labor Day, Columbus Day, Veterans Day, Thanks...","[Takes Reservations, Offers Delivery, Offers T...",369.0,2.9,41.0,(925) 778-3845,None,None,None,$11-30,"[114, 51, 45, 62, 97]",eba22f6c961d84fc919bf8a62f47cc52ac1ad534c824e5...,4350 Century Blvd,114.0,51.0,45.0,62.0,97.0,restaurants,CA,Pittsburg,Outback Steakhouse,"[American, Steakhouse]",38.00588,-121.84010,"4350 Century Blvd, Pittsburg, CA 94565",None,(925) 778-3845,https://www.facebook.com/OutbackSteakhouse.Pit...,3.5,41.0,4.0,[],1.0,4.0,11.0,15.0,10.0,3.5,4.5,3.5,4.0,True,False,"Outback Steakhouse, the home of juicy steaks, ...",$$ - $$$,America/Los_Angeles,None,None,None,None,None,None,None,[],"[Dinner, Lunch]","[Seating, Highchairs Available, Wheelchair Acc...",Mid-range,Sit down,dfae318ebb270c4bbf79d9ec8e79a28e7943bc2e17f7e4...,4350 Century Blvd,True
24,7eec46f26e89fea7bbf809a21da7b7ac93731f44de14ad...,c497d6f7304e9cd3edd84aa0e4db8881fc2d7bd588ac70...,b121e2391acb7fde4632d195ef9dfc3cb2dc8ca089e547...,30333c1afb2c4e17b9576da98021d4c8f6b14bd7e322d6...,CA,Palo Alto,Tacolicious,"[Caterers, Tacos]",37.44332,-122.16103,"632 Emerson St , Palo Alto, CA 94301",None,https://www.tacolicious.com/menu/eat/,https://www.tacolicious.com/location/palo-alto/,False,True,ALREADY_CLAIMED,11:00 AM - 9:00 PM,11:00 AM - 9:00 PM,11:00 AM - 9:00 PM,11:00 AM - 9:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 9:00 PM,"[Labor Day, Columbus Day, Veterans Day, Thanks...","[Takes Reservations, Offers Delivery, Offers T...",951.0,3.4,48.0,None,None,Tacolicious is a Cal-Mex restaurant with a San...,2009,$11-30,"[110, 136, 205, 256, 244]",b121e2391acb7fde4632d195ef9dfc3cb2dc8ca089e547...,632 Emerson St,110.0,136.0,205.0,256.0,244.0,restaurants,CA,Palo Alto,Tacolicious,"[Mexican, Latin, Spanish, Gluten Free Options]",37.44332,-122.16103,"632 Emerson St, Palo Alto, CA 94301-1609",None,(415) 649-6077,http://www.tacolicious.com/location/palo-alto,4.0,95.0,3.0,[],2.0,6.0,13.0,34.0,36.0,4.0,4.0,3.5,4.0,

(567, 86)

(0, 86)

In [11]:
# creating dictionat for having same type of data like area\,latitude .......
#creating two dictionary one for categorical value second for int float type
# for categorical if none is present in our parent table we will fill up by second
# for numerical we will put maximum of them
# for list type we will union them keeping unique of them
# removing unnenecasry columns
columns_to_drop=[]
columns_to_fill = {
    'subregion_yan': 'subregion_tan',
    'area_yan': 'area_tan',
    'city_yan':'city_tan',
    'brand_name_yan':'brand_name_tan',
    'latitude_yan':'latitude_tan',
    'longitude_yan':'longitude_tan',
    'address_yan':'address_tan',
    'website_url':'restaurant_website',
    'is_claimed':'claimed',
    'telephone_number_yan':'telephone_number_tan',
    'description_yan':'description_tan',
    'operating_hours_mon_yan':'operating_hours_mon_tan',
    'operating_hours_tue_yan':'operating_hours_tue_tan',
    'operating_hours_wed_yan':'operating_hours_wed_tan',
    'operating_hours_thu_yan':'operating_hours_thu_tan',
    'operating_hours_fri_yan':'operating_hours_fri_tan',
    'operating_hours_sat_yan':'operating_hours_sat_tan',
    'operating_hours_sun_yan':'operating_hours_sun_tan'

}
columns_to_drop = ['place_features_tan','address_tan','rating_tan','very_good_review_count_tan','review_count_tan','excellent_review_count_tan','average_review_count_tan','poor_review_count_tan','terrible_review_count_tan','operating_hours_sun_tan','operating_hours_sat_tan','operating_hours_fri_tan','operating_hours_thu_tan','operating_hours_wed_tan','operating_hours_tue_tan','operating_hours_mon_tan','description_tan','telephone_number_tan','claimed','subregion_tan', 'area_tan','city_tan','brand_name_tan','categories_tan','latitude_tan','longitude_tan','restaurant_website']
int_values={  'terrible_review_count_yan':'terrible_review_count_tan',
    'poor_review_count_yan':'poor_review_count_tan',
    'average_review_count_yan':'average_review_count_tan',
    'very_good_review_count_yan':'very_good_review_count_tan',
    'excellent_review_count_yan':'excellent_review_count_tan',
    'review_count_yan':'review_count_tan',
    'rating_yan':'rating_tan',
     'very_good_review_count_yan':'very_good_review_count_tan'         }
# Iterate over the columns to fill
for col_to_fill, col_source in columns_to_fill.items():
    columns_to_drop.append(col_source)
    merged_df[col_to_fill].fillna(merged_df[col_source], inplace=True)

for col_to_fill,col_source in int_values.items():
  columns_to_drop.append(col_source)
  merged_df[col_to_fill] = np.where(
    (merged_df[col_to_fill].isna() & merged_df[col_source].isna()),  # Check if both are None
    None,  # Set col_to_fill to None if both are None
    np.where(
        (merged_df[col_to_fill].isna()) | (merged_df[col_to_fill] < merged_df[col_source]),
        merged_df[col_source],
        merged_df[col_to_fill]
    )
)


merged_df['categories_yan'] = merged_df.apply(lambda row: union_and_process_lists(row['categories_yan'], row['categories_tan']), axis=1)
merged_df['palce_feature_yan'] = merged_df.apply(lambda row: union_and_process_lists(row['place_features_yan'], row['place_features_tan']), axis=1)
columns_to_drop.extend(["categories_tan","place_features_tan"])
merged_df.drop(columns=columns_to_drop, inplace=True)

### second merging on delta

In [12]:
#left merge on table_e on table_c and then on table_d
merged_df = merged_df.merge(table_d, left_on='delta', right_on='id', how='left',suffixes=("","_del"))

In [13]:
# looking for same latitude and longitude and filtering out if
filtered_df = merged_df[(merged_df['id_tan'].notna()) & (merged_df['id'].notna())]
filtered_df = filtered_df[filtered_df['id_tan'].isin(filtered_df['id'])]
display(filtered_df.shape)
filtered_df = merged_df[(merged_df['id_yan'].notna()) & (merged_df['id'].notna())]
filtered_df = filtered_df[filtered_df['id_yan'].isin(filtered_df['id'])]
display(filtered_df.shape)

(0, 82)

(0, 82)

In [14]:
# creating dictionat for having same type of data like area\,latitude .......
#creating two dictionary one for categorical value second for int float type
# for categorical if none is present in our parent table we will fill up by second
# for numerical we will put maximum of them
# for list type we will union them keeping unique of them
# removing unnenecasry columns
column_to_drop=[]
columns_to_fill = {
    'industry':'industry_del',
    'subregion_yan': 'subregion',
    'area_yan': 'area',
    'city_yan':'city',
    'brand_name_yan':'brand_name',
    'latitude_yan':'latitude',
    'longitude_yan':'longitude',
    'address_yan':'address'


}
for col_to_fill, col_source in columns_to_fill.items():
    column_to_drop.append(col_source)
    merged_df[col_to_fill].fillna(merged_df[col_source], inplace=True)
merged_df['categories_yan'] = merged_df.apply(lambda row: union_and_process_lists(row['categories_yan'], row['categories']), axis=1)
merged_df['palce_features_yan'] = merged_df.apply(lambda row: union_and_process_lists(row['place_features_yan'], row['palce_feature_yan']), axis=1)


int_values={
    'rating_yan':'rating'
}
for col_to_fill,col_source in int_values.items():
  column_to_drop.append(col_source)
  merged_df[col_to_fill] = np.where(
    (merged_df[col_to_fill].isna() & merged_df[col_source].isna()),  # Check if both are None
    None,  # Set col_to_fill to None if both are None
    np.where(
        (merged_df[col_to_fill].isna()) | (merged_df[col_to_fill] < merged_df[col_source]),
        merged_df[col_source],
        merged_df[col_to_fill]
    )
)

column_to_drop.extend(["categories","palce_feature_yan"])


merged_df.drop(columns=column_to_drop, inplace=True)

### last merging on oscar

In [15]:
merged_df = merged_df.merge(table_a, left_on='oscar', right_on='id', how='left',suffixes=("","_osc"))

In [16]:
# creating dictionat for having same type of data like area\,latitude .......
#creating two dictionary one for categorical value second for int float type
# for categorical if none is present in our parent table we will fill up by second
# for numerical we will put maximum of them
# for list type we will union them keeping unique of them
# removing unnenecasry columns
column_to_drop=[]
columns_to_fill = {
    'subregion_yan': 'subregion',
    'area_yan': 'area',
    'city_yan':'city',
    'brand_name_yan':'brand_name',
    'latitude_yan':'latitude',
    'longitude_yan':'longitude',
    'address_yan':'address',
    'description_yan':'description',
    'website_url':'restaurant_website',
    'telephone_number_yan':'phone_number',


    'offers_delivery':'has_delivery_partners',
    'offers_pickup':'has_pickup',
    'restaurant_provided_menu_link':'menu_url',
    'operating_hours_mon_yan':'operating_hours_mon',
    'operating_hours_tue_yan':'operating_hours_tue',
    'operating_hours_wed_yan':'operating_hours_wed',
    'operating_hours_thu_yan':'operating_hours_thu',
    'operating_hours_fri_yan':'operating_hours_fri',
    'operating_hours_sat_yan':'operating_hours_sat',
    'operating_hours_sun_yan':'operating_hours_sun',
    'cross_streets':'cross_street'
}

int_values={
    'rating_yan':'rating',
    'rating_count':'rating_count_osc',
    'atmosphere_rating':'atmosphere_rating_osc',
    'food_rating':'food_rating_osc',
    'service_rating':'service_rating_osc',
    'value_rating':'value_rating_osc',
    'terrible_review_count_yan':'terrible_review_count',
    'poor_review_count_yan':'poor_review_count',
    'average_review_count_yan':'average_review_count',
    'very_good_review_count_yan':'very_good_review_count',
    'excellent_review_count_yan':'excellent_review_count',

}
for col_to_fill, col_source in columns_to_fill.items():
    merged_df[col_to_fill].fillna(merged_df[col_source], inplace=True)
    column_to_drop.append(col_source)

for col_to_fill,col_source in int_values.items():
  column_to_drop.append(col_source)
  merged_df[col_to_fill] = np.where(
    (merged_df[col_to_fill].isna() & merged_df[col_source].isna()),  # Check if both are None
    None,  # Set col_to_fill to None if both are None
    np.where(
        (merged_df[col_to_fill].isna()) | (merged_df[col_to_fill] < merged_df[col_source]),
        merged_df[col_source],
        merged_df[col_to_fill]
    )
)
merged_df['categories_yan'] = merged_df.apply(lambda row: union_and_process_lists(row['categories_yan'], row['categories']), axis=1)

column_to_drop.extend(["categories","review_count_not_recommended"])

In [17]:
merged_df.drop(columns=column_to_drop,inplace=True)

## how did i mapping for that  i have helping code which I have attached a cell link , I have first merged each data and then multi hod encoded our range and then performed some analysis in which

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Sample data



# # Multi-hot encode the price range values
# price_range_encoded = pd.get_dummies(df[['price_range_a', 'price_range_b', 'price_range_c', 'price_range_d']])

# # Concatenate the one-hot encoded values with the original DataFrame
# df_encoded = pd.concat([df, price_range_encoded], axis=1)

# # Group by price_range_id and calculate the sum of each one-hot encoded column
# grouped_data = df_encoded.groupby('price_range_id').sum()

# # Display the grouped data
# print(grouped_data)
# sums=grouped_data.sum()
# per_data = (grouped_data/sums)*100
# print(per_data)


# colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'tab:blue', 'tab:orange',
#           'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink',
#           'tab:gray', 'tab:olive', 'tab:cyan']

# # Create scatterplots for each column with different colors
# for i, col in enumerate(per_data.columns):
#     plt.scatter(per_data.index, per_data[col], label=col, color=colors[i])

# # Add labels and legend
# plt.xlabel('Index')
# plt.ylabel('Value')
# plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))

# # Show the plot
# plt.show()

In [18]:
# here we have mapped our price_range to common on analysis
reversed_mapping = {
    "Inexpensive": "$11-30",
    "$11-30": "$11-30",
    "$31-60": "$31-60",
    "Pricey": "$31-60",
    "$$": "$11-30",
    "Moderate": "$11-30",
    "Ultra High-End": "Above $61",
    "Above $61": "Above $61",
    "Under $10": "$11-30",
    "$30 and under":"$11-30",
    "$31 to $50":"$31-60",
    "$50 and over":"Above $61",
    "$":"$11-30",
    "$$ - $$$":"$31-60",
    "$$$$":"Above $61",
    "$$$":"$31-60"
}
columns_to_map = ['price_range_yan', 'price_range_tan', 'price_range', 'price_range_osc']
def map_values(row):
    for column in columns_to_map:
        if row[column] is not None:
            return reversed_mapping.get(row[column])

merged_df['mapped_range'] = merged_df.apply(map_values, axis=1)
columns_to_map.append("mapped_range")

In [ ]:
# handling operating_hours from table_b as it was not so conclusive
selected_columns = ['operating_hours_mon_yan', 'operating_hours_sun_yan', 'operating_hours_tue_yan', 'operating_hours_wed_yan', 'operating_hours_thu_yan', 'operating_hours_fri_yan', 'operating_hours_sat_yan']

# Check if all selected columns are None for each row
mask = merged_df[selected_columns].isna().all(axis=1)

# Fill the selected columns with the corresponding values from 'operating_hours' for rows where all selected columns are None
merged_df.loc[mask, selected_columns] = merged_df.loc[mask, 'operating_hours']
merged_df['description_yan'].fillna(merged_df['brand_description'], inplace=True)

In [19]:
column_to_drop=["operating_timezone","is_network_non_bookable","is_permanently_closed","has_proof_of_vaccination_outdoor","price_range_id","is_convenience_store","offers_package_returns","offers_cannabis","awards","oscar","tango","delta","yankee","id_yan","review_count_by_rating","industry","id_tan","is_valid_email","id","operating_hours_osc","id_osc",'price_range_yan', 'price_range_tan', 'price_range', 'price_range_osc','brand_description','operating_hours']
merged_df.reset_index()
merged_df.drop(columns=column_to_drop,inplace=True)

In [20]:
for col in merged_df.columns:
    if col.endswith('yan'):
        new_col_name = col.replace('_yan', '')
        merged_df.rename(columns={col: new_col_name}, inplace=True)

In [21]:
# removing any duplicate if brand_name and address are same
# giving id
result_df = merged_df[merged_df.duplicated(subset=['brand_name', 'address'], keep=False)]
print(result_df.shape)
merged_df=merged_df.drop_duplicates(subset=['brand_name', 'address'], keep='first')
merged_df = merged_df.drop_duplicates(subset=['brand_name', 'latitude','longitude'], keep='first')
merged_df=merged_df.rename(columns={"facebook":"facebook_link"})
merged_df['id'] = range(1, len(merged_df) + 1)

(22, 97)


In [22]:
merged_df.head(3)

,subregion,city,brand_name,categories,latitude,longitude,address,cross_streets,restaurant_provided_menu_link,website_url,is_claimable,is_claimed,claimability_reason,operating_hours_mon,operating_hours_tue,operating_hours_wed,operating_hours_thu,operating_hours_fri,operating_hours_sat,operating_hours_sun,observed_holidays,place_features,review_count,rating,telephone_number,description,specialties,year_established,area,terrible_review_count,poor_review_count,average_review_count,very_good_review_count,excellent_review_count,email_address,ranking,food_rating,service_rating,value_rating,atmosphere_rating,is_closed,dietary_restrictions,meals_offered,place_type_by_price,place_type_by_service,rating_count,offers_delivery,offers_pickup,offers_group_order,offers_catering,palce_features,public_transit,primary_cuisine,dining_style,executive_chef_name,parking_info,dress_code,entertainment,payment_options,maximum_days_advance_for_reservation,noise_rating,most_recent_review,review_count,review_topics,tags,has_clean_menus,has_common_area_cleaning,has_common_area_distancing,has_contact_tracing_collected,has_contactless_payment,requires_diner_temperature_check,has_limited_seating,prohibits_sick_staff,requires_proof_of_vaccination,requires_diner_masks,requires_wait_staff_masks,has_sanitized_surfaces,provides_sanitizer_for_customers,has_sealed_utensils,has_vaccinated_staff,requires_staff_temp_checks,has_table_layout_with_extra_space,is_waitlist_only,has_waitlist,has_bar,has_counter,has_high_top_seating,has_outdoor_seating,has_priority_seating,has_private_dining,has_takeout,has_gifting,order_online_link,delivery_partners,facebook_link,daily_reservation_count,mapped_range,id
0,CA,Pittsburg,Outback Steakhouse,"[Appetizers, Seafood, Dessert, Steakhouse, Chi...",38.00588,-121.84010,"4350 Century Blvd , Pittsburg, CA 94565",None,https://www.outback.com/menu,https://locations.outback.com/california/pitts...,False,True,ALREADY_CLAIMED,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 10:00 PM,11:30 AM - 11:00 PM,11:30 AM - 11:00 PM,11:30 AM - 9:00 PM,"[Labor Day, Columbus Day, Veterans Day, Thanks...","[Takes Reservations, Offers Delivery, Offers T...",369.0,4.3,(925) 778-3845,"Outback Steakhouse, the home of juicy steaks, ...",None,None,4350 Century Blvd,114.0,51.0,45.0,62.0,97.0,None,4.0,3.5,4.5,3.5,4.0,False,[],"[Dinner, Lunch]",Mid-range,Sit down,7500.0,True,True,True,False,"[Seating, Highchairs Available, Wheelchair Acc...",None,Steak,Casual Dining,None,Public Lot,Casual Dress,None,"[AMEX, Diners Club, Discover, Mastercard, Visa]",90.0,0.0,None,0.0,[],"[Delivery, Takeout]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,None,[],None,NaN,$11-30,1
1,CA,Menlo Park,Naomi Sushi,"[Japanese, Asian, Pickup, Sushi, Takeout]",37.45560,-122.18642,"1328 El Camino Real , Menlo Park, CA 94025",NaN,NaN,http://www.naomisushi.com,True,True,RECLAIMABLE,,5:00 PM - 8:30 PM,5:00 PM - 8:30 PM,5:00 PM - 8:30 PM,5:00 PM - 8:30 PM,5:00 PM - 8:30 PM,5:00 PM - 8:30 PM,"[Labor Day, Columbus Day, Veterans Day, Thanks...","[Takes Reservations, Offers Delivery, Offers T...",291.0,4.7,(650) 321-6902,NaN,None,None,1328 El Camino Real,51.0,38.0,51.0,76.0,75.0,None,1.0,4.5,4.5,3.5,4.5,False,[],"[Dinner, Lunch]",Mid-range,Sit down,1800.0,True,True,True,False,"[Takeout, Serves Alcohol, Reservations, Seatin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$11-30,2
2,CA,Hayward,Famous Dave's Bar-B-Que,"[Burgers, Soup, Barbeque, Comfort Food, Sandwi...",37.65401,-122.10663,"790 W Winton Ave , Hayward, CA 94545",Southland Pl & Stonewall Ave,https://www.famousdaves.com/Menu,https://www.famousdaves.com/hayward-ca,False,True,ALREADY_CLAIMED,11:00 AM - 8:00 PM,11:00 AM - 8:00 PM,11:00 AM - 8:00 PM,11:00 AM - 9:00 PM,11:00 AM - 9:

## Section 3: Web Harvesting 🕸 [15]

<img src="https://media.giphy.com/media/wSeaJHrOckToQ/giphy.gif" width="350px" alt="merge">

In this section, your task is to enhance the current dataset by navigating to the restaurant websites specified in the data, retrieving any incomplete information, and collecting additional features that could be essential for the analysis. This will involve accessing the URLs of restaurant websites listed in the dataset, and potentially exploring linked pages from the initial page to gather required information. The depth of pages you explore is at your discretion, as long as the added information substantially benefits the final dataset.

Tasks include:

- **Filling Missing Data**: Identify and fill in missing values in the dataset, such as restaurant descriptions, operating hours, and contact information.
- **Extracting New Features**: Extract additional valuable information from the restaurant websites, such as:
  - **Social Media Handles**: Links to Facebook, Twitter, Instagram, LinkedIn pages.
  - **Awards and Recognitions**: Any awards or recognitions received by the restaurant.
  - **Special Events or Offers**: Information on any special events, promotions, or discounts.
  - Anything else that you feel may be relevant and valuable

Be respectful of the website's `robots.txt` file and terms of service, and ensure your web scraping script is efficient, accurate, and does not overload the servers. You will be graded on the efficiency and accuracy of your script, as well as the completeness and usefulness of the extracted data. Ensure to update the dataset without altering its existing structure and content.

> You may need to additionally join and clean up the resultant dataset

## function

1. categorize_links()\
Function to categorize links on a webpage using string matching

2. findPhone()\
Function to scrap phone number from webpages using regex for matching

3. findEmail()\
Function to scrap email from webpages using regex for matching

4. findBooleanValues()\
Function to scrap values which are stored as True/False from webpages using regex for matching

5. findYear()\
Function to scrap opened year from webpage usign regex for matching

6. findTime()\
Function to scrap open and closing time from webpage using regex for matching

7. extract_links_with_error_handling()\
Function to crawl and extract links, any other useful data from each link upto 'max_depth' and store it in output file

8. crawl_link()\
Function to match strings in links to get various links

9. write_output()\
Function to write all info in an output file

In [ ]:
def categorize_links(data_dict, links):
    if links is None:
        return []
    other_links = []
    for link in links:
        if "menu" in link:
            data_dict["restaurant_provided_menu_link"] = link
        elif "order" in link:
            data_dict["order_online_link"] = link
        elif "facebook" in link:
            data_dict["facebook_link"] = link
        elif "instagram" in link:
            data_dict["instagram_link"] = link
        elif "twitter" in link:
            data_dict["twitter_link"] = link
        elif "play_store" in link:
            data_dict["play_store_link"] = link
        elif "app_store" in link:
            data_dict["app_store_link"] = link
        elif "maps" in link:
            data_dict["maps_link"] = link
        else:
            other_links.append(link)

def findPhone(soup):
  phone_pattern = r'\+\d{1,2}\s?\(\d{3}\)\s?\d{3}[-\s]\d{4}'
  phones = []
  pattern_elements = soup.find_all(string=re.compile(phone_pattern))
  for element in pattern_elements:
      stripped_ele = element.strip()
      if stripped_ele is not None:
          stripped_ele = stripped_ele.strip()
      match = re.search(phone_pattern, stripped_ele)
      phones.append(match.group())
  return phones

def findEmail(soup):
  email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
  pattern_elements = soup.find_all(string=re.compile(email_pattern))
  for element in pattern_elements:
      stripped_ele = element.strip()
      if stripped_ele is not None:
          stripped_ele = stripped_ele.strip()
      match = re.search(email_pattern, stripped_ele)
      return match.group()

def findBooleanValues(soup):
    keywords_pattern = ["pickup", "group", "catering", "counter", "bar", "delivery", "gifting", "takeout"]
    def findPattern(keyword):
        if soup.find_all(string=re.compile(r'\b(?:' + keyword + r')\b', re.IGNORECASE)):
            return True
        return False
    dict = {}
    for keyword in keywords_pattern:
        dict[keyword] = findPattern(keyword)
    return dict

def findYear(soup):
    year_pattern = r'\b\d{4}\b'
    pattern_elements = soup.find_all(string=re.compile(year_pattern))
    possible_year = []
    for element in pattern_elements:
        stripped_ele = element.strip()
        possible_year.append(stripped_ele)
    all_phones = findPhone(soup)
    possible_year = [ele for ele in possible_year if not any(phn in ele for phn in all_phones)]
    possible_year = [year for year in possible_year if '©' not in year]
    actually_possible_years = []
    for year in possible_year:
      match = re.search(year_pattern, year)
      actually_possible_years.append(int(match.group()))
    for yr in actually_possible_years:
      if(yr > datetime.now().year):
        actually_possible_years.remove(yr)
    return actually_possible_years

def findTime(soup):
  time_pattern = r'\b\d{1,2}(?::\d{2})?\s*[APap][Mm]\b'
  pattern_elements = soup.find_all(string=re.compile(time_pattern, re.IGNORECASE))
  for element in pattern_elements:
      parent_element = element.find_parent(attrs={"class": True})
      matches = parent_element.get_text()
      formatted_data = ' '.join(matches.split())
  return formatted_data


import file
we are using two dictionary to dynamically call only function corresponding our dictionary have none value on each link we will call we are only traversing till 1 level \
used threading to increase our efficiency\
i mostly worked on colab thus have different , however i will be adding input and output file

In [ ]:
input_filename = 'input/input2.csv'
output_filename = 'output/output2.csv'

link_data_lock = threading.Lock()

def write_output(data_dict, output_filename):
    header = list(data_dict.keys())
    with open(output_filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        if csvfile.tell() == 0:
            writer.writeheader()
        writer.writerow(data_dict)

# Function to crawl a single link and store the result in link_data
def crawl_link(link_index, link, max_depth, data_dict):
    visited_links = set()
    result = extract_links_with_error_handling(
        link,
        link_index,
        data_dict,
        visited_links,
        depth=0,
        max_depth=max_depth
    )

    categorize_links(data_dict, result)
    write_output(data_dict, output_filename)

    with link_data_lock:
        link_data[link] = result

def extract_links_with_error_handling(url, index, data_dict, visited_links=None, depth=0, max_depth=2):
    if visited_links is None:
        visited_links = set()

    # retries = 3  # Number of retries before giving up
    # retry_delay = 2  # Delay between retries in seconds
    unique_links = set()

    try:
        if url not in visited_links and depth <= max_depth:
            visited_links.add(url)
            response = requests.get(url, timeout=(5, 5))
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            for (key, value) in data_dict.items():
                if(value == ''):
                    if(key ==  'telephone_number'):
                        data_dict[key] = findPhone(soup)
                    if(key == 'year_established'):
                        data_dict[key] = findYear(soup)
                    if(key == 'email_address'):
                        data_dict[key] = findEmail(soup)
                    if( key == 'offers_delivery' or
                        key == 'offers_pickup' or
                        key == 'offers_group_order' or
                        key == 'offers_catering' or
                        key == 'has_bar' or
                        key == 'has_counter' or
                        key == 'has_gifting' or
                        key == 'has_takeout' ):
                        for(key2, val2) in findBooleanValues(soup).items():
                            if(key2 in key and value == ''):
                                data_dict[key] = val2

            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]
            for link in valid_links:
                if not any(social in link for social in ["whatsapp", "insta", "facebook", "twitter"]):
                    unique_links.add(link)

    except Timeout:
        print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
        # time.sleep(retry_delay)
    except RequestException as e:
        print(f"RequestException occurred for index {index} and URL {url}: {str(e)}")
    except Exception as e:
        print(f"Error extracting links from index {index} for URL {url}: {str(e)}")

    return unique_links

# Initialize an empty dictionary to store the data
link_data = {}
threads = []

with open(input_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    dummy_dict = {}
    header_row = next(reader)
    for key in header_row:
        dummy_dict[key] = None
    new_rows = ['instagram_link', 'twitter_link', 'play_store_link', 'app_store_link']
    for row in new_rows:
        dummy_dict[row] = None

    for index, row in enumerate(reader):
        if len(row) < 2:
            continue
        link_index = row[0]
        link = row[1]
        data_dict = {}
        if(link == ''):
            continue

        for key in dummy_dict.keys():
            if key in header_row:
                data_dict[key] = row[header_row.index(key)]
            else:
                data_dict[key] = None

        visited_links = set()
        max_depth = 1
        thread = threading.Thread(target=crawl_link, args=(link_index, link, max_depth, data_dict))
        thread.start()
        threads.append(thread)
        time.sleep(1)

# Wait for all threads to finish
for thread in threads:
    thread.join()

 above code will work on personal system,  we are sending particular columns from our dataframe i have taken to not overload scrapping

 id,website_url,restaurant_provided_menu_link,telephone_number,year_established,email_address,offers_delivery,offers_pickup,offers_group_order,offers_catering,has_bar,has_counter,has_gifting,has_takeout,order_online_link,facebook_link


it will return id,website_url,restaurant_provided_menu_link,telephone_number,year_established,email_address,offers_delivery,offers_pickup,offers_group_order,offers_catering,has_bar,has_counter,has_gifting,has_takeout,order_online_link,facebook_link,instagram_link,twitter_link,play_store_link,app_store_link,maps_link,extra


for above helping code

In [ ]:
out1=pd.read_csv("combined.csv")
display(out1.shape)
out1.info()
merged_df = merged_df.merge(out1, left_on='id', right_on='id', how='inner',suffixes=("","_out1"))

column_to_drop=[]
columns_to_fill = {
    'restaurant_provided_menu_link': 'restaurant_provided_menu_link_out1',
    'telephone_number': 'telephone_number_out1',
    'email_address':'email_address_out1',
    'offers_delivery':'offers_delivery_out1',
    'offers_pickup':'offers_pickup_out1',
    'offers_group_order':'offers_group_order_out1',
    'offers_catering':'offers_catering_out1',
    'has_bar':'has_bar_out1',
    'has_counter':'has_counter_out1',
    'has_gifting':'has_gifting_out1',
    'has_takeout':'has_takeout_out1',
    'order_online_link':'order_online_link_out1',


}

for col_to_fill, col_source in columns_to_fill.items():
    merged_df[col_to_fill].fillna(merged_df[col_source], inplace=True)
    column_to_drop.append(col_source)
def handling_year(val1,list1):
  if val1 is not None:
    return val1

  else:
    if isinstance(list1,float):
      return list1
    if list1 is None or len(list1)==0 or len(list1)>1:
      return None
    else:
      return list1[0]
  return val1

merged_df['year_established'] = merged_df.apply(lambda row: handling_year(row['year_established'], row['year_established_out1']), axis=1)


column_to_drop.extend(["year_established_out1","website_url_out1","id"])
merged_df.drop(columns=column_to_drop,inplace=True)

## Section 4: Data Quality Checks ✅ [5]

<img src="https://media.giphy.com/media/NS7gPxeumewkWDOIxi/giphy.gif" width="350px" alt="merge">

In this section, you are expected to devise and execute a series of data quality tests on the dataframe. Your objective is to identify and address potential edge cases that could affect the downstream use of the dataset. You may use [Great Expectations](https://greatexpectations.io/) or any other tool of your preference to run the tests and profile the data.


- **Test Design**: Outline the data quality tests you plan to run. Explain why you have chosen these tests and what potential issues they could uncover.
- **Test Execution**: Execute the designed tests on the dataframe. Document the results of each test, including any discrepancies or anomalies identified.
- **Data Profiling**: Perform a thorough profiling of the data to understand its characteristics and quality. This may include understanding the distribution of different features, identifying outliers, or assessing the completeness and uniqueness of the data.
- **Edge Case Identification**: Discuss any edge cases you have identified during the testing and profiling process. Explain how these edge cases could affect the downstream use of the dataset and propose potential solutions to address them.

> 📝 Your ability to design comprehensive tests, identify and address edge cases, and thoroughly profile the data will be crucial in this section. Be sure to document your process and findings clearly and concisely

In [ ]:
!pip install great-expectations

In [ ]:
merged_df.to_csv("testing.csv")
import great_expectations as ge
df_1=ge.read_csv("testing.csv")
df_1.expect_column_to_exist("rating")
df_1.expect_column_distinct_values_to_be_in_set('mapped_range',['$11-30','$31-60','Above $61'])
df_1.expect_column_values_to_be_in_set('is_closed',value_set=[True,False])
df_1.expect_column_values_to_match_regex('year_established',r'\b\d{4}\b')
df_1.expect_column_unique_value_count_to_be_between('year_established',min_value=1900,max_value=2023)
df_1.expect_column_values_to_be_between('rating',max_value= 10,min_value=1)
df_1.expect_column_values_to_match_regex('operating_hours_sun',r'\d{1,2}:\d{2} [APap][Mm] - \d{1,2}:\d{2} [APap][Mm](?: , \d{1,2}:\d{2} [APap][Mm] - \d{1,2}:\d{2} [APap][Mm])*')
df_1.expect_column_values_to_match_regex('operating_hours_sat',r'\d{1,2}:\d{2} [APap][Mm] - \d{1,2}:\d{2} [APap][Mm](?: , \d{1,2}:\d{2} [APap][Mm] - \d{1,2}:\d{2} [APap][Mm])*')
df_1.expect_column_values_to_be_in_set(
    column="is_claimed",  # The name of the column you want to validate
    value_set=[True, False],  # List of expected valid values
    mostly=True
)
df_1.expect_column_values_to_match_regex("website_url",r'^(https?://)[^\s/$.?#].[^\s]*$')
df_1.expect_column_values_to_match_regex("facebook_link",r'^(https?://)?(www\.)?facebook\.com/[A-Za-z0-9_.-]+/?$')
df_1.expect_column_values_to_match_regex("telephone_number",r'^\(\d{3}\) \d{3}-\d{4}$')
df_1.expect_column_values_to_match_regex("restaurant_provided_menu_link",r'^(https?://)?(www\.)?([a-zA-Z0-9-]+)(\.[a-z]{2,4})(/[a-zA-Z0-9-_.~%]*)*([?][a-zA-Z0-9-_.~%&=]*)?$')
df_1.expect_column_values_to_be_of_type('address',"object")
df_1.expect_column_value_lengths_to_be_between('subregion',2,3)
df_1.expect_column_mean_to_be_between('latitude',34,38)
df_1.expect_column_mean_to_be_between('longitude',-119,-122)


Save your results in a parquet file and upload them to your private fork of the GitHub Repository.